# All the Imports

In [1]:
import sys
import os
import logging
sys.path.insert(1, '../src/')

In [2]:
%%time 
import itertools 
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import LabelEncoder
from lassonet import LassoNetClassifier


from RankCorr.picturedRocks import Rocks
import smashpy as smashpy


import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F

from utils import *


CPU times: user 7.98 s, sys: 763 ms, total: 8.74 s
Wall time: 16.5 s


In [3]:
from os import path

In [4]:
model_save_path = '../data/{}/models/'
num_times = 10

  

In [5]:
data_locations = [
                  'CITE-seq',
                 # 'developmental_mouse',
                  'mouse_brain_broad',
                  'paul',
                  'zeisel'
                 # 'synthetic'
]

In [6]:
result_axis = [
 ('All Markers', 'all_markers_results_{}.npy', 'all_markers_knn_results_{}.npy'), 
 ('L1 VAE', 'l1_vae_results_{}.npy', 'l1_vae_knn_results_{}.npy'),
 ('Smash RF', 'smash_rf_results_{}.npy', 'smash_rf_knn_results_{}.npy'),
 ('SMaSH DNN', 'smash_results_{}.npy', 'smash_knn_results_{}.npy'),
 ('RankCorr', 'rankcorr_results_{}.npy', 'rankcorr_knn_results_{}.npy'),
 ('Global Gate VAE', 'globalgate_vae_results_{}.npy', 'globalgate_vae_knn_results_{}.npy'),
 ('MarkerMap Unsupervised', 'marker_map_unsupervised_results_{}.npy', 'marker_map_unsupervised_knn_results_{}.npy'),
 ('MarkerMap Supervised', 'marker_map_supervised_results_{}.npy', 'marker_map_supervised_knn_results_{}.npy'),
 ('MarkerMap Mixed Supervision', 'marker_map_mixed_results_{}.npy', 'marker_map_mixed_knn_results_{}.npy'),
 ('Concrete VAE',  'concrete_vae_results_{}.npy', 'concrete_vae_knn_results_{}.npy'),
 ('LassoNet', 'lasso_net_results_{}.npy', 'lasso_net_knn_results_{}.npy')
]

## Mean Accuracies and F1

### Random Forest

In [7]:
results = {}
for e, axis in itertools.product(data_locations, result_axis):
    m, f1, _, _ = model_variances(model_save_path.format(e) + axis[1], num_times)
    m = round(m, 3)
    f1 = round(f1, 3)
    if axis[0] in results:
        results[axis[0]][e] = (m, f1)
    else:
        results[axis[0]] = {}
        results[axis[0]][e] = (m, f1)

tabulated_results1 = pd.DataFrame(results)
tabulated_results1

,All Markers,L1 VAE,Smash RF,SMaSH DNN,RankCorr,Global Gate VAE,MarkerMap Unsupervised,MarkerMap Supervised,MarkerMap Mixed Supervision,Concrete VAE,LassoNet
CITE-seq,"(0.077, 0.9)","(0.119, 0.849)","(0.067, 0.917)","(0.071, 0.911)","(0.065, 0.918)","(0.132, 0.834)","(0.085, 0.893)","(0.062, 0.924)","(0.065, 0.919)","(0.13, 0.835)","(0.066, 0.918)"
mouse_brain_broad,"(0.009, 0.991)","(0.148, 0.842)","(0.02, 0.98)","(0.014, 0.986)","(0.031, 0.969)","(0.093, 0.901)","(0.024, 0.975)","(0.006, 0.994)","(0.012, 0.987)","(0.143, 0.847)","(0.009, 0.991)"
paul,"(0.184, 0.796)","(0.532, 0.399)","(0.119, 0.877)","(0.17, 0.817)","(0.144, 0.851)","(0.389, 0.566)","(0.142, 0.855)","(0.118, 0.879)","(0.14, 0.856)","(0.393, 0.563)","(0.121, 0.873)"
zeisel,"(0.046, 0.953)","(0.207, 0.777)","(0.047, 0.952)","(0.082, 0.915)","(0.05, 0.949)","(0.207, 0.779)","(0.114, 0.879)","(0.047, 0.952)","(0.07, 0.927)","(0.227, 0.756)","(0.05, 0.948)"


In [8]:
tabulated_results1[['All Markers', 'Smash RF', 'RankCorr', 'MarkerMap Supervised', 'Concrete VAE', 'LassoNet']]

,All Markers,Smash RF,RankCorr,MarkerMap Supervised,Concrete VAE,LassoNet
CITE-seq,"(0.077, 0.9)","(0.067, 0.917)","(0.065, 0.918)","(0.062, 0.924)","(0.13, 0.835)","(0.066, 0.918)"
mouse_brain_broad,"(0.009, 0.991)","(0.02, 0.98)","(0.031, 0.969)","(0.006, 0.994)","(0.143, 0.847)","(0.009, 0.991)"
paul,"(0.184, 0.796)","(0.119, 0.877)","(0.144, 0.851)","(0.118, 0.879)","(0.393, 0.563)","(0.121, 0.873)"
zeisel,"(0.046, 0.953)","(0.047, 0.952)","(0.05, 0.949)","(0.047, 0.952)","(0.227, 0.756)","(0.05, 0.948)"


### KNN

In [9]:
results = {}
for e, axis in itertools.product(data_locations, result_axis):
    m, f1, _, _ = model_variances(model_save_path.format(e) + axis[2], num_times)
    m = round(m, 3)
    f1 = round(f1, 3)
    if axis[0] in results:
        results[axis[0]][e] = (m, f1)
    else:
        results[axis[0]] = {}
        results[axis[0]][e] = (m, f1)

tabulated_results2 = pd.DataFrame(results)
tabulated_results2

,All Markers,L1 VAE,Smash RF,SMaSH DNN,RankCorr,Global Gate VAE,MarkerMap Unsupervised,MarkerMap Supervised,MarkerMap Mixed Supervision,Concrete VAE,LassoNet
CITE-seq,"(0.097, 0.898)","(0.164, 0.812)","(0.069, 0.919)","(0.078, 0.909)","(0.073, 0.915)","(0.189, 0.786)","(0.1, 0.888)","(0.062, 0.929)","(0.069, 0.92)","(0.18, 0.797)","(0.064, 0.925)"
mouse_brain_broad,"(0.099, 0.889)","(0.228, 0.75)","(0.03, 0.969)","(0.015, 0.985)","(0.041, 0.959)","(0.14, 0.848)","(0.028, 0.972)","(0.005, 0.995)","(0.016, 0.984)","(0.223, 0.752)","(0.009, 0.991)"
paul,"(0.321, 0.608)","(0.537, 0.392)","(0.212, 0.777)","(0.291, 0.676)","(0.247, 0.743)","(0.447, 0.521)","(0.256, 0.735)","(0.218, 0.77)","(0.263, 0.726)","(0.447, 0.521)","(0.231, 0.754)"
zeisel,"(0.056, 0.944)","(0.283, 0.701)","(0.046, 0.953)","(0.104, 0.893)","(0.049, 0.95)","(0.283, 0.703)","(0.152, 0.84)","(0.043, 0.957)","(0.073, 0.924)","(0.303, 0.68)","(0.055, 0.944)"


In [10]:
tabulated_results2[['All Markers', 'Smash RF', 'RankCorr', 'MarkerMap Supervised', 'Concrete VAE', 'LassoNet']]

,All Markers,Smash RF,RankCorr,MarkerMap Supervised,Concrete VAE,LassoNet
CITE-seq,"(0.097, 0.898)","(0.069, 0.919)","(0.073, 0.915)","(0.062, 0.929)","(0.18, 0.797)","(0.064, 0.925)"
mouse_brain_broad,"(0.099, 0.889)","(0.03, 0.969)","(0.041, 0.959)","(0.005, 0.995)","(0.223, 0.752)","(0.009, 0.991)"
paul,"(0.321, 0.608)","(0.212, 0.777)","(0.247, 0.743)","(0.218, 0.77)","(0.447, 0.521)","(0.231, 0.754)"
zeisel,"(0.056, 0.944)","(0.046, 0.953)","(0.049, 0.95)","(0.043, 0.957)","(0.303, 0.68)","(0.055, 0.944)"


## Variances

### Random Forest

In [11]:
results = {}
for e, axis in itertools.product(data_locations, result_axis):
    _, _, m, f1 = model_variances(model_save_path.format(e) + axis[1], num_times)
    m = round(m, 5)
    f1 = round(f1, 5)
    if axis[0] in results:
        results[axis[0]][e] = (m, f1)
    else:
        results[axis[0]] = {}
        results[axis[0]][e] = (m, f1)

tabulated_results1 = pd.DataFrame(results)
tabulated_results1

,All Markers,L1 VAE,Smash RF,SMaSH DNN,RankCorr,Global Gate VAE,MarkerMap Unsupervised,MarkerMap Supervised,MarkerMap Mixed Supervision,Concrete VAE,LassoNet
CITE-seq,"(0.00435, 0.00524)","(0.02712, 0.03567)","(0.00579, 0.00716)","(0.00614, 0.00804)","(0.00795, 0.0105)","(0.01802, 0.02309)","(0.01126, 0.01384)","(0.00473, 0.00686)","(0.0033, 0.00443)","(0.01472, 0.02009)","(0.00356, 0.00498)"
mouse_brain_broad,"(0.00104, 0.00113)","(0.04504, 0.04978)","(0.00603, 0.00603)","(0.00085, 0.00094)","(0.00314, 0.00315)","(0.02766, 0.0289)","(0.03373, 0.03433)","(0.00092, 0.00096)","(0.0023, 0.00241)","(0.02385, 0.02509)","(0.00092, 0.00093)"
paul,"(0.01397, 0.01625)","(0.03251, 0.033)","(0.01065, 0.01174)","(0.02674, 0.03112)","(0.01679, 0.01694)","(0.02522, 0.03058)","(0.01845, 0.01891)","(0.01424, 0.01522)","(0.01348, 0.01391)","(0.0203, 0.02298)","(0.01134, 0.01176)"
zeisel,"(0.0053, 0.00556)","(0.04539, 0.05313)","(0.00912, 0.00924)","(0.04131, 0.04437)","(0.0068, 0.00703)","(0.02734, 0.03039)","(0.04135, 0.0462)","(0.00706, 0.00728)","(0.00863, 0.01026)","(0.03839, 0.04548)","(0.00562, 0.00588)"


In [12]:
tabulated_results1[['All Markers', 'Smash RF', 'RankCorr', 'MarkerMap Supervised', 'Concrete VAE', 'LassoNet']]

,All Markers,Smash RF,RankCorr,MarkerMap Supervised,Concrete VAE,LassoNet
CITE-seq,"(0.00435, 0.00524)","(0.00579, 0.00716)","(0.00795, 0.0105)","(0.00473, 0.00686)","(0.01472, 0.02009)","(0.00356, 0.00498)"
mouse_brain_broad,"(0.00104, 0.00113)","(0.00603, 0.00603)","(0.00314, 0.00315)","(0.00092, 0.00096)","(0.02385, 0.02509)","(0.00092, 0.00093)"
paul,"(0.01397, 0.01625)","(0.01065, 0.01174)","(0.01679, 0.01694)","(0.01424, 0.01522)","(0.0203, 0.02298)","(0.01134, 0.01176)"
zeisel,"(0.0053, 0.00556)","(0.00912, 0.00924)","(0.0068, 0.00703)","(0.00706, 0.00728)","(0.03839, 0.04548)","(0.00562, 0.00588)"


### KNN

In [13]:
results = {}
for e, axis in itertools.product(data_locations, result_axis):
    _, _, m, f1 = model_variances(model_save_path.format(e) + axis[2], num_times)
    m = round(m, 5)
    f1 = round(f1, 5)
    if axis[0] in results:
        results[axis[0]][e] = (m, f1)
    else:
        results[axis[0]] = {}
        results[axis[0]][e] = (m, f1)

tabulated_results2 = pd.DataFrame(results)
tabulated_results2

,All Markers,L1 VAE,Smash RF,SMaSH DNN,RankCorr,Global Gate VAE,MarkerMap Unsupervised,MarkerMap Supervised,MarkerMap Mixed Supervision,Concrete VAE,LassoNet
CITE-seq,"(0.00808, 0.00723)","(0.04667, 0.05552)","(0.00627, 0.0064)","(0.0074, 0.0093)","(0.00499, 0.00659)","(0.02023, 0.02436)","(0.01409, 0.01397)","(0.00515, 0.00624)","(0.00568, 0.00581)","(0.02569, 0.02945)","(0.00446, 0.0058)"
mouse_brain_broad,"(0.00792, 0.00776)","(0.05017, 0.05611)","(0.00994, 0.01014)","(0.00224, 0.00233)","(0.00438, 0.0044)","(0.04504, 0.05125)","(0.03682, 0.03773)","(0.00073, 0.00073)","(0.00497, 0.00512)","(0.03372, 0.03833)","(0.00173, 0.00176)"
paul,"(0.01681, 0.02255)","(0.04479, 0.03331)","(0.01405, 0.01586)","(0.01848, 0.02353)","(0.02077, 0.0194)","(0.02577, 0.02474)","(0.02663, 0.02721)","(0.01787, 0.01939)","(0.01766, 0.01783)","(0.03495, 0.03453)","(0.01122, 0.01493)"
zeisel,"(0.00823, 0.00832)","(0.0444, 0.04644)","(0.00764, 0.0078)","(0.06605, 0.06672)","(0.00615, 0.00612)","(0.03307, 0.0335)","(0.06278, 0.06798)","(0.007, 0.00709)","(0.01154, 0.0121)","(0.03226, 0.03649)","(0.00653, 0.00682)"


In [14]:
tabulated_results2[['All Markers', 'Smash RF', 'RankCorr', 'MarkerMap Supervised', 'Concrete VAE', 'LassoNet']]

,All Markers,Smash RF,RankCorr,MarkerMap Supervised,Concrete VAE,LassoNet
CITE-seq,"(0.00808, 0.00723)","(0.00627, 0.0064)","(0.00499, 0.00659)","(0.00515, 0.00624)","(0.02569, 0.02945)","(0.00446, 0.0058)"
mouse_brain_broad,"(0.00792, 0.00776)","(0.00994, 0.01014)","(0.00438, 0.0044)","(0.00073, 0.00073)","(0.03372, 0.03833)","(0.00173, 0.00176)"
paul,"(0.01681, 0.02255)","(0.01405, 0.01586)","(0.02077, 0.0194)","(0.01787, 0.01939)","(0.03495, 0.03453)","(0.01122, 0.01493)"
zeisel,"(0.00823, 0.00832)","(0.00764, 0.0078)","(0.00615, 0.00612)","(0.007, 0.00709)","(0.03226, 0.03649)","(0.00653, 0.00682)"
